In [ ]:
import cv2
import pandas as pd
import numpy as np
import glob
import re
import shutil
import os
import subprocess

# Extract info from CSV filenames
a = []
b = []
for file in glob.glob("PatternExtract/PatternExtract-Ki67/CSV/*"):
    b.append(file)
    bit0 = file.split("_1_HP_IM3_0_Core")[0].split(" ")[-1]
    
    bit1 = file.split("_1_HP_IM3_0_Core")[1].split("_")[0].split(",")
    
    p3 = f"{int(bit1[1]):02d}"
    p4 = f"{int(bit1[2]):02d}"
    
    a.append([bit0, p3, p4])

# Extract info from image filenames
c = []
d = []
for file in glob.glob("PatternExtract/PatternExtract-Ki67/scripts/R_ki67/Images/*"):
    d.append(file)
    bit2 = file.split("_1_HP_IM3_0_Core")[0].split(" ")[-1]
    
    bit3 = file.split("_1_HP_IM3_0_Core")[1].split("_")[0].split(",")
    
    p1 = f"{int(bit3[1]):02d}"
    p2 = f"{int(bit3[2]):02d}"
    
    c.append([bit2, p1, p2])

# Print the first parsed image result (if available)
if c and d:
    print(c[0])
    print(d[0])
else:
    print("No image files found.")


In [ ]:
for j, item in enumerate(a):  # a[j] is ID info from CSV
    try:
        i = c.index(item)  # Match to image identifier
    except ValueError:
        print(f"No match for: {item}")
        continue

    image3 = cv2.imread(d[i])
    image4 = image3.copy()

    df = pd.read_csv(b[j], sep='\t')

    print("CSV:", b[j])
    print("Image:", d[i])

    # First overlay: red dot at centroid
    for row in range(df.shape[0]):
        x = int(df.loc[row, "Centroid X µm"]) * 2
        y = int(df.loc[row, "Centroid Y µm"]) * 2
        cv2.circle(image3, (x, y), radius=0, color=[0, 0, 255], thickness=15)

    # Second overlay: larger circle
    for row in range(df.shape[0]):
        x = int(df.loc[row, "Centroid X µm"]) * 2
        y = int(df.loc[row, "Centroid Y µm"]) * 2
        cv2.circle(image4, (x, y), radius=5, color=[0, 0, 200], thickness=15)

    # Blend overlays
    dst_0 = cv2.addWeighted(image3, 0.5, image4, 0.5, 0)

    # Save output
    outname = os.path.basename(b[j]) + ".tiff"
    cv2.imwrite(f"PatternExtract/PatternExtract-Ki67/scripts/Mask/{outname}", dst_0)


In [ ]:
##Windows OS

import os
import subprocess

# Step 1: Change to the QuPath installation directory (where the console .exe is located)
os.chdir("C:/Program Files/QuPath-0.3.2")  # Update this to your actual QuPath install path

# Step 2: Define the Groovy script that creates the QuPath project and adds images
create_project_script = "C:/Users/sridharg/PatternExtract/PatternExtract-Ki67/scripts/createproject_ki67.groovy"

# Step 3: Define the path to the image directory (to populate into the project)
image_dir = "C:/Users/sridharg/PatternExtract/PatternExtract-Ki67/scripts/Mask"

# Step 4: Run the project creation script using QuPath's console mode
subprocess.run([
    "QuPath-0.3.2 (console).exe",  # This is the Windows console executable
    "script",
    "--args", image_dir,
    create_project_script
])

# Step 5: Define the annotation script and the QuPath project file
annotate_script = "PatternExtract/PatternExtract-Ki67/scripts/test_ki67.groovy"
project_file = "PatternExtract/PatternExtract-Ki67/Project/ki67/project.qpproj"

# Step 6: Run the annotation script, export results (e.g., GeoJSON), and save the project
subprocess.run([
    "QuPath-0.3.2 (console).exe",
    "script",
    "--save",
    "--project", project_file,
    annotate_script
])

# Step 7: Optionally launch QuPath GUI with the created project
subprocess.Popen([
    "QuPath-0.3.2 (console).exe",
    "--project", project_file
])


In [ ]:
##Mac OS

import os
import subprocess

# Step 1: Change working directory to QuPath installation directory (macOS .app path)
os.chdir("/Applications/QuPath.app/Contents/MacOS/")  # Adjust if QuPath is elsewhere

# Path to Groovy script that creates the QuPath project and adds images
create_project_script = "PatternExtract/PatternExtract-Ki67/scripts/createproject_ki67.groovy"

# Directory containing the images to be added to the QuPath project
image_dir = "/PatternExtract/PatternExtract-Ki67/scripts/Mask"

# Step 2: Run Groovy script to create and populate the QuPath project
subprocess.run([
    "./QuPath",  # Use actual QuPath binary on macOS
    "script",
    "--args", image_dir,
    create_project_script
])

# Step 3: Define the annotation script and the QuPath project file
annotate_script = "/PatternExtract/PatternExtract-Ki67/scripts/test_ki67.groovy"
project_file = "/PatternExtract/PatternExtract-Ki67/Project/ki67/project.qpproj"

# Step 4: Run the annotation script, save the project (generates GeoJSON, etc.)
subprocess.run([
    "./QuPath",
    "script",
    "--save",
    "--project", project_file,
    annotate_script
])

# Step 5: Optionally open the project in QuPath GUI for inspection
subprocess.Popen([
    "./QuPath",
    "--project", project_file
])


In [ ]:
import subprocess

##WINDOWS OS

# Path to the R script that performs downstream analysis (e.g., spatial stats, plotting)
Rfile = "/PatternExtract/PatternExtract-Ki67/scripts/R_script_ki67.R"

# Full path to Rscript executable (update if your R version or install path differs)
rscript_exe = "C:/Program Files/R/R-4.3.1/bin/Rscript.exe"

# Run the R script with '--vanilla' to avoid loading user environment or site files
subprocess.call([rscript_exe, "--vanilla", Rfile])


In [ ]:
import subprocess

##MacOS

# Path to the R script you want to execute
Rfile = "/PatternExtract/PatternExtract-Ki67/scripts/R_script_ki67.R"

# Path to Rscript binary (standard for R installations on macOS via CRAN)
rscript_exe = "/usr/local/bin/Rscript"  # or run `which Rscript` in terminal to confirm

# Run the R script with '--vanilla' for a clean R session
subprocess.call([rscript_exe, "--vanilla", Rfile])
